# 1) 데이터 준비와 전처리

In [1]:
import pandas as pd

import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)


In [5]:
movies_nunique = ratings['movie_id'].nunique()
users_nunique = ratings['user_id'].nunique()
top_30_movies = ratings.groupby('movie_id')['user_id'].count().sort_values(ascending=False).head(30)


print(movies_nunique, users_nunique)
print(top_30_movies)

3628 6039
movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64


# 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.


In [6]:
ratings

my_ratings = pd.DataFrame({
    'user_id': [6041] * 5,
    'movie_id': [1, 2, 3, 4, 5],
    'count': [3, 4, 5, 1, 2],
    'timestamp': [6, 7, 8, 9, 10],
})

ratings_new = ratings.append(my_ratings)

ratings_new.tail(10)

ratings_new.shape, ratings.shape

((836483, 4), (836478, 4))

# 4) CSR matrix를 직접 만들어 봅시다.


In [7]:
from scipy.sparse import csr_matrix

csr_data = csr_matrix((ratings_new['count'], (ratings_new['user_id'], ratings_new['movie_id'])))

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


In [8]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [9]:
csr_data_transpose = csr_data.T
als_model.fit(csr_data_transpose)

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


In [10]:
my_ratings

,user_id,movie_id,count,timestamp
0,6041,1,3,6
1,6041,2,4,7
2,6041,3,5,8
3,6041,4,1,9
4,6041,5,2,10


In [11]:
def checkMovie(user_id, movie_id):
    user_factor, movie_factor = als_model.user_factors[user_id], als_model.item_factors[movie_id]
    
    return np.dot(user_factor, movie_factor)

checkMovie(my_ratings.iloc[0]['user_id'], my_ratings.iloc[0]['movie_id'])

0.32160914

In [12]:
checkMovie(1, 4)

0.02020276

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.


In [13]:
def get_movies_by_ids(movie_ids):
    return movies[movies['movie_id'].isin(movie_ids)]

similar_movie_ids = [i[0] for i in als_model.similar_items(my_ratings.iloc[0]['movie_id'])]
get_movies_by_ids(similar_movie_ids)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
33,34,Babe (1995),Children's|Comedy|Drama
360,364,"Lion King, The (1994)",Animation|Children's|Musical
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
591,595,Beauty and the Beast (1991),Animation|Children's|Musical
1245,1265,Groundhog Day (1993),Comedy|Romance
1854,1923,There's Something About Mary (1998),Comedy
2252,2321,Pleasantville (1998),Comedy
2286,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy


# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [14]:
recommend_movie_ids = [i[0] for i in als_model.recommend(my_ratings.iloc[0]['user_id'], csr_data)]
get_movies_by_ids(recommend_movie_ids)

,movie_id,title,genre
33,34,Babe (1995),Children's|Comedy|Drama
59,60,"Indian in the Cupboard, The (1995)",Adventure|Children's|Fantasy
314,317,"Santa Clause, The (1994)",Children's|Comedy|Fantasy
363,367,"Mask, The (1994)",Comedy|Crime|Fantasy
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
778,788,"Nutty Professor, The (1996)",Comedy|Fantasy|Romance|Sci-Fi
2286,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy
3381,3450,Grumpy Old Men (1993),Comedy
3420,3489,Hook (1991),Adventure|Fantasy
